In [ ]:
import pandas as pd
import numpy as np
from utils import plot_cm
import matplotlib.pyplot as plt
from pydantic import BaseModel

In [ ]:
path = '/Users/Alex/Desktop/2.FutureTech/OSFM/uniform_sample/results/9-13-and-8-28-with-confidence.csv'
df = pd.read_csv(path)

uses_questions = [10, 0, 13, 11, 23, 20, 8, 28, 3, 18, 15, 1, 2, 21, 17, 22, 4, 9, 14, 5, 16, 25, 19, 6, 26, 29, 27, 24, 52, 12]
extends_questions = [50, 30, 40, 31, 41, 51, 38, 42, 35, 32, 52, 48, 54, 58, 53, 33, 34, 44, 36, 37, 47, 56, 43, 1, 8, 12, 18, 22, 57, 3]

In [ ]:
df['answer_vector_np'] = df['answer_vector'].apply(lambda s: np.array(s[1:-1].split(' ')))
df['extends_score'] = df['answer_vector_np'].apply(lambda s: (s[extends_questions]).astype(int).sum()/len(extends_questions))
df['uses_score'] = df['answer_vector_np'].apply(lambda s: (s[uses_questions]).astype(int).sum()/len(uses_questions))

In [ ]:
class Citation(BaseModel):
    citation: str# Literal['context', 'uses', 'extends']
def try_json(s):
    s = s.replace('```','').replace('json', '')
    try:
        return Citation.model_validate_json(s).citation
    except Exception as e:
        print(e)
        return None
df['postprocess_reclass_stripped'] =df['postprocess_reclass'].apply(try_json)

In [ ]:
model_column = 'gpt4omini_prompt1_10'
df['manual_classification'] = ['extends' if e_score >= .3 else 'uses' if u_score >= .3 else 'context' for e_score, u_score in zip(df['extends_score'], df['uses_score'])]
df['corrected_classification'] = [reclass if c in {'uses','extends'} else c for reclass, c in zip(df['postprocess_reclass_stripped'], df['manual_classification'])]
df['corrected_classification'] = df['corrected_classification'].apply(lambda s: {'background':'context'}.get(s) or s)

In [ ]:
ignore_mask = (df['multisentence'].apply(lambda s: s.find('\\begin{tab') != -1)) | (df['strippedModelKey'] == 'ase') | df['corrected_classification'].isna()
df_valid = df[~ignore_mask]

In [ ]:
plot_cm(df_valid['alex'], df_valid['corrected_classification'])
#plt.title(model_column)

In [ ]:
None == None

In [ ]:
df['gpt4o_prompt1_10']

In [ ]:
import matplotlib.pyplot as plt

column = 'llama31_prompt1_10'
plt.hist(df[column])
plt.title(column)